In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
import sys
sys.path.append('../scripts') 
from functions import *
from config import nutrient_info, conversion_factors

In [2]:
df_food = pd.read_csv("../data/df_food_filtered_with_food_group.csv",index_col=0)
with open("../data/df_food_openfoodfacts_merged.pkl", "rb") as f:
    df_food_openfoodfacts = pickle.load(f, encoding="latin1")

C:\Users\Sophie\AppData\Local\Temp\ipykernel_21176\2857174756.py:1: DtypeWarning: Columns (2,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food = pd.read_csv("../data/df_food_filtered_with_food_group.csv",index_col=0)


In [3]:
df_food = df_food[df_food["eaten_quantity_in_gram"] > 0]
df_food = df_food[df_food["energy_kcal_eaten"] > 0]

In [4]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}



In [5]:
df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_21176\2711278687.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')


In [6]:
df_food_scaled[['QI', 'DI','NB', 'combined_name', 'food_group']].groupby('food_group').median(numeric_only=True).sort_values('QI')

,QI,DI,NB
food_group,,,
alcoholic_beverages,0.038127,0.007668,3.812709
sweetened_beverages,0.041806,0.664300,4.180602
others,0.105016,0.666667,5.474351
soups_sauces_condiments,0.117561,1.068889,6.402525
salty_snacks,0.121828,0.535766,10.000000
cooked_grains_pasta,0.135563,0.517566,9.358974
sweets_desserts,0.157373,0.852030,12.639102
vegetable_oils,0.245871,0.694173,6.645221
butter_margarine_spreads,0.284294,1.837749,15.794434


In [7]:
df_meat_fish_seafood = df_food_scaled[df_food_scaled['food_group'] == 'meat_fish_seafood']

In [8]:
df_meat_fish_seafood.drop_duplicates('combined_name')

,food_id,barcode,dish_id,eaten_quantity,eaten_unit,subject_key,eaten_at,eaten_at_utc_offset,media_count,food_group_cname,...,protein_eaten_ratio_scaled,fat_eaten_ratio_scaled,fatty_acids_saturated_eaten_ratio_scaled,cholesterol_eaten_ratio_scaled,sugar_eaten_ratio_scaled,sodium_eaten_ratio_scaled,salt_eaten_ratio_scaled,QI,DI,NB
0,13,0,335994,110.0,g,bjsqab,2022-02-15 18:06:27,60,1,dairy_products_meat_fish_eggs_tofu,...,7.936971,0.774393,1.037218,3.937360,0.000000,0.430649,0.000000,2.498497,1.029937,67.110660
56,27,0,118524,60.0,g,nwfrgc,2020-05-11 17:53:04,120,1,dairy_products_meat_fish_eggs_tofu,...,6.306737,1.183432,1.448551,2.783883,0.008791,22.756410,0.000000,6.850892,4.696845,72.655733
800,71,0,372924,100.0,g,7wuem4,2022-07-06 18:42:58,120,1,dairy_products_meat_fish_eggs_tofu,...,4.889424,1.570402,2.359232,2.029988,0.000000,0.129758,0.000000,1.260086,1.014897,53.767267
802,82,0,102980,120.0,g,cbqxxj,2020-03-23 11:04:30,60,1,dairy_products_meat_fish_eggs_tofu,...,7.555239,0.868712,1.490313,2.295082,0.000000,0.223133,0.000000,1.708448,0.812873,56.628417
815,83,0,113397,250.0,g,8vtq3c,2020-04-27 17:30:00,120,0,dairy_products_meat_fish_eggs_tofu,...,8.051530,0.743906,0.953984,3.497942,0.000000,0.303498,0.000000,3.356512,0.916555,60.032115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317129,33144,7613312222911,386534,155.0,g,x4xuqb,2022-09-22 15:21:21,120,0,unclassified,...,6.769071,1.074774,1.088732,0.000000,0.047904,0.000000,1.896208,0.338454,0.684603,5.000000
317360,33417,7613269437260,393863,300.0,g,vxyht3,2022-10-21 20:15:55,120,0,unclassified,...,5.521049,0.712251,0.721501,0.000000,0.825397,0.000661,0.000000,0.291925,0.376635,6.587302
317488,33609,3660088100350,400544,90.0,g,7mm74m,2022-11-16 11:25:53,60,0,unclassified,...,3.801242,1.846154,1.641558,0.000000,0.036571,0.000000,0.761905,0.190062,0.714365,5.000000
317589,46072,7613312352717,406386,125.0,g,48wbxy,2022-12-08 11:10:36,60,0,unclassified,...,3.881988,1.831502,1.217532,0.000000,0.000000,0.000000,0.148810,0.194099,0.532974,5.000000


In [9]:
high_qi_meat_fish_seafood = df_meat_fish_seafood[df_meat_fish_seafood['QI'] > 10].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [10]:
high_qi_meat_fish_seafood[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
260724,Perch fillets (lake),120.0,61.567363,1.045119,80.933707
154859,Liver,30.0,31.765218,3.232951,77.415456
294168,"Sushi Nigiri ""saumon & thon""",90.0,17.293963,0.484705,62.241389
276920,GRAN MAR Polypfisch in Marinade,57.5,12.878941,1.905372,82.178683
285020,Octopus,110.0,12.093442,1.193991,81.319930


In [11]:
high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Perch fillets (lake)'][[i for i in df_meat_fish_seafood.columns if '_ratio_scaled' in i]].T

,260724
folate_eaten_ratio_scaled,0.672269
niacin_eaten_ratio_scaled,3.577431
pantothenic_acid_eaten_ratio_scaled,2.722689
vitamin_b2_eaten_ratio_scaled,1176.470588
vitamin_b1_eaten_ratio_scaled,1.260504
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.000000
vitamin_b12_eaten_ratio_scaled,5.812325
vitamin_b6_eaten_ratio_scaled,1.680672
vitamin_c_eaten_ratio_scaled,0.851541


In [12]:
food_item = high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Perch fillets (lake)'].iloc[0]
qi_including = food_item['QI']
results = []

for nutr in qualifying_nutrients:
    qi_excluding = compute_qi_excluding(food_item, qualifying_nutrients, exclude=nutr, scaling_factor=2000)
    abs_diff = qi_including - qi_excluding
    pct_diff = (abs_diff / qi_including) * 100
    results.append({
        'Nutrient Excluded': nutr,
        'QI (incl.)': qi_including,
        'QI (excl.)': qi_excluding,
        'Absolute Change': abs_diff,
        'Percent Change': pct_diff
    })


df_results = pd.DataFrame(results)
df_results.sort_values('Absolute Change', ascending=False)

,Nutrient Excluded,QI (incl.),QI (excl.),Absolute Change,Percent Change
3,vitamin_b2_eaten,61.567363,3.048704,58.518659,95.048182
9,vitamin_d_eaten,61.567363,67.287733,-5.720370,-9.291238
18,protein_eaten,61.567363,67.959596,-6.392233,-10.382502
6,vitamin_b12_eaten,61.567363,68.085274,-6.517911,-10.586634
1,niacin_eaten,61.567363,68.209435,-6.642072,-10.788300
2,pantothenic_acid_eaten,61.567363,68.256921,-6.689558,-10.865428
14,phosphorus_eaten,61.567363,68.268125,-6.700762,-10.883627
7,vitamin_b6_eaten,61.567363,68.314810,-6.747447,-10.959455
16,zinc_eaten,61.567363,68.318312,-6.750949,-10.965142
10,vitamin_e_activity_eaten,61.567363,68.326015,-6.758652,-10.977654


We observe that for the 'Perch fillets (lake)', the vitamin b2 is the only micronutrient that boost the QI value

In [13]:
high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Liver'][[i for i in df_meat_fish_seafood.columns if '_ratio_scaled' in i]].T

,154859
folate_eaten_ratio_scaled,7.621951
niacin_eaten_ratio_scaled,13.066202
pantothenic_acid_eaten_ratio_scaled,20.000000
vitamin_b2_eaten_ratio_scaled,32.150776
vitamin_b1_eaten_ratio_scaled,2.540650
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,242.160279
beta_carotene_eaten_ratio_scaled,0.000000
vitamin_b12_eaten_ratio_scaled,279.471545
vitamin_b6_eaten_ratio_scaled,8.255159
vitamin_c_eaten_ratio_scaled,3.414634


In [14]:
food_item = high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Liver'].iloc[0]
qi_including = food_item['QI']
results = []

for nutr in qualifying_nutrients:
    qi_excluding = compute_qi_excluding(food_item, qualifying_nutrients, exclude=nutr, scaling_factor=2000)
    abs_diff = qi_including - qi_excluding
    pct_diff = (abs_diff / qi_including) * 100
    results.append({
        'Nutrient Excluded': nutr,
        'QI (incl.)': qi_including,
        'QI (excl.)': qi_excluding,
        'Absolute Change': abs_diff,
        'Percent Change': pct_diff
    })


df_results = pd.DataFrame(results)
df_results.sort_values('Absolute Change', ascending=False)

,Nutrient Excluded,QI (incl.),QI (excl.),Absolute Change,Percent Change
6,vitamin_b12_eaten,31.765218,19.768490,11.996728,37.766869
5,all_trans_retinol_equivalents_activity_eaten,31.765218,21.841338,9.923880,31.241341
3,vitamin_b2_eaten,31.765218,33.508532,-1.743314,-5.488124
2,pantothenic_acid_eaten,31.765218,34.183576,-2.418358,-7.613225
1,niacin_eaten,31.765218,34.568787,-2.803569,-8.825907
7,vitamin_b6_eaten,31.765218,34.836067,-3.070849,-9.667331
0,folate_eaten,31.765218,34.871245,-3.106027,-9.778075
16,zinc_eaten,31.765218,34.938996,-3.173778,-9.991361
18,protein_eaten,31.765218,34.942678,-3.177460,-10.002953
12,iron_eaten,31.765218,34.970989,-3.205771,-10.092079


Same thing, the vitamin b12 and the RAE are the two micronutrients that influence the most the QI value of 'Liver'

In [15]:
high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Sushi Nigiri "saumon & thon"'][[i for i in df_meat_fish_seafood.columns if '_ratio_scaled' in i]].T

,294168
folate_eaten_ratio_scaled,0.387435
niacin_eaten_ratio_scaled,1.323859
pantothenic_acid_eaten_ratio_scaled,0.879581
vitamin_b2_eaten_ratio_scaled,266.539743
vitamin_b1_eaten_ratio_scaled,0.523560
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.629768
vitamin_b12_eaten_ratio_scaled,5.453752
vitamin_b6_eaten_ratio_scaled,1.772050
vitamin_c_eaten_ratio_scaled,0.273647


In [16]:
food_item = high_qi_meat_fish_seafood[high_qi_meat_fish_seafood['combined_name'] == 'Sushi Nigiri "saumon & thon"'].iloc[0]
qi_including = food_item['QI']
results = []

for nutr in qualifying_nutrients:
    qi_excluding = compute_qi_excluding(food_item, qualifying_nutrients, exclude=nutr, scaling_factor=2000)
    abs_diff = qi_including - qi_excluding
    pct_diff = (abs_diff / qi_including) * 100
    results.append({
        'Nutrient Excluded': nutr,
        'QI (incl.)': qi_including,
        'QI (excl.)': qi_excluding,
        'Absolute Change': abs_diff,
        'Percent Change': pct_diff
    })


df_results = pd.DataFrame(results)
df_results.sort_values('Absolute Change', ascending=False)

,Nutrient Excluded,QI (incl.),QI (excl.),Absolute Change,Percent Change
3,vitamin_b2_eaten,17.293963,4.372764,12.921199,74.715084
10,vitamin_e_activity_eaten,17.293963,15.728908,1.565055,9.049717
6,vitamin_b12_eaten,17.293963,18.877541,-1.583578,-9.156826
7,vitamin_b6_eaten,17.293963,19.082080,-1.788117,-10.339545
18,protein_eaten,17.293963,19.082518,-1.788555,-10.342076
14,phosphorus_eaten,17.293963,19.106564,-1.812601,-10.481120
1,niacin_eaten,17.293963,19.106980,-1.813017,-10.483523
9,vitamin_d_eaten,17.293963,19.118476,-1.824513,-10.549998
2,pantothenic_acid_eaten,17.293963,19.131662,-1.837699,-10.626244
4,vitamin_b1_eaten,17.293963,19.151441,-1.857478,-10.740613


Vitamin b2 is the one with the most influence